In [1]:
import syft as sy
from syft.service.job.job_stash import JobStatus


In [2]:
node = sy.orchestra.launch(
    name="test",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=c7abfb54f31547f3b7bab8f891282d46 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
@sy.syft_function_single_use()
def test():
    return 1

SyftSuccess: Syft function 'test' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
domain_client.code.request_code_execution(test)

syft.service.request.request.Request

In [6]:
domain_client.requests[-1].approve()

Approving request for domain test


SyftSuccess: Request bc0c4c244e014c4189a80a0101c1d6ea changes applied

In [7]:
domain_client.code[0].id

<UID: f41e6790364e490387d55619d3c24f4a>

In [8]:
for i in range(5):
    domain_client.api.services.job.create_job_for_user_code_id(domain_client.code[0].id)

In [9]:
from syft.types.uid import UID


def update_job(job):
    job.status = JobStatus.PROCESSING
    job.current_iter = int(76252 * 0.83)
    job.n_iters = 76252
    domain_client.api.services.job.update(job)
    return job
    
def kill_job(job):
    job.status = JobStatus.INTERRUPTED
    job.current_iter = 1
    job.n_iters = 76252
    domain_client.api.services.job.update(job)

def complete_job(job):
    job.status = JobStatus.COMPLETED
    job.current_iter = 76252
    job.n_iters = 76252
    domain_client.api.services.job.update(job)
    uid = job.log_id
    domain_client.api.services.log.append(uid, new_str='/opt/conda/lib/python3.10/site-packages/traitlets/traitlets.py:2930: FutureWarning: --Exporter.preprocessors=["nbconvert.preprocessors.ExtractOutputPreprocessor"] for containers is deprecated in traitlets 5.0. You can pass `--Exporter.preprocessors item` ... multiple times to add items to a list.\n')
    domain_client.api.services.log.append(uid, new_str='  warn( \n')
    domain_client.api.services.log.append(uid, new_str='[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`. \n')
    domain_client.api.services.log.append(uid, new_str='[NbConvertApp] Converting notebook __notebook__.ipynb to html \n')
    domain_client.api.services.log.append(uid, new_str='[NbConvertApp] Writing 474651 bytes to __results__.html\n')


def failed_job(job):
    job.status = JobStatus.ERRORED
    job.current_iter = int(76252 * 0.2)
    job.n_iters = 76252
    domain_client.api.services.job.update(job)

def idle_job(job):
    domain_client.api.services.job.update(job)

In [10]:
complete_job(domain_client.jobs[0])
update_job(domain_client.jobs[1])
failed_job(domain_client.jobs[2])
kill_job(domain_client.jobs[3])
idle_job(domain_client.jobs[4])

In [11]:
domain_client.jobs[0].logs()

/opt/conda/lib/python3.10/site-packages/traitlets/traitlets.py:2930: FutureWarning: --Exporter.preprocessors=["nbconvert.preprocessors.ExtractOutputPreprocessor"] for containers is deprecated in traitlets 5.0. You can pass `--Exporter.preprocessors item` ... multiple times to add items to a list.
  warn( 
[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`. 
[NbConvertApp] Converting notebook __notebook__.ipynb to html 
[NbConvertApp] Writing 474651 bytes to __results__.html




In [12]:
domain_client.jobs[0].logs()

/opt/conda/lib/python3.10/site-packages/traitlets/traitlets.py:2930: FutureWarning: --Exporter.preprocessors=["nbconvert.preprocessors.ExtractOutputPreprocessor"] for containers is deprecated in traitlets 5.0. You can pass `--Exporter.preprocessors item` ... multiple times to add items to a list.
  warn( 
[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`. 
[NbConvertApp] Converting notebook __notebook__.ipynb to html 
[NbConvertApp] Writing 474651 bytes to __results__.html




In [13]:
domain_client.jobs[0]

Time,#,Message
0.0s,0,"/opt/conda/lib/python3.10/site-packages/traitlets/traitlets.py:2930: FutureWarning: --Exporter.preprocessors=[""nbconvert.preprocessors.ExtractOutputPreprocessor""] for containers is deprecated in traitlets 5.0. You can pass `--Exporter.preprocessors item` ... multiple times to add items to a list."
0.0s,1,warn(
0.0s,2,[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
0.0s,3,[NbConvertApp] Converting notebook __notebook__.ipynb to html
0.0s,4,[NbConvertApp] Writing 474651 bytes to __results__.html
0.0s,5,


In [14]:
domain_client.api.node_name

'test'

TODO .jobs:
* add adjustable columns
* ~~add default `--`~~
* ~~what do to if worker does not exists? Just dont show it~~
* improve styles for progress and eta
* improve logs trimming and style
* **add filters** -> Ask Kyoko/Thiago for prio
* yellow badge for processing
* 

TODO job_repr:
* add blue header (tricky for subjobs)
* ~~add `created_by` to get user name and email~~
* ~~add `finished_at`~~ -> changed into updated_at
* replace mock result
* ~~fix padding for logs~~
* ~~fix color highlighting for tabs~~
* ~~add lines under tabs~~
* add time for logs -> Ask Kyoko
* remove different line background (maybe not?) -> Ask Kyoko

General TODOS:
* test with real jobs
* add repr for subjobs and logs
* allow users to use string for UIDs
* add all the api methods that use copy button

Ask Kyoko:
* do we really want logs to be separated in lines or by timestamps?
* remove the worker pool column?
* do we want a copy button on job repr for worker pool ids as well?
* what is the id in the job's usercode's name?

* 10 GBs